In [1]:
# latest update
# April 26, 2017
# Changes done by: Jimmy Vivas
# Added code description

# This code was developed by Jimmy Vivas

# The code will take all CSv files from a specific directory and process them at the same Time
# The code will get replace strings in the values for numeric data
#   It will assign an integer value for each distinct string for each column (sensor)
#   The integer value will be reset it for each sensor
#   The code does not evaluare wether or not the sensor should stay. THe analyst will make this decision
#   Using Previse or any other tool
# The code will split the csv file if the result is more than 1MM rows

In [2]:
from os import listdir
import pandas as pd
import math
import numpy as np

In [3]:
# mac Path
#CSVPath1 =  '/Users/jvivas/Documents/XMT Baytwon Sensor Data' \
#            '/Sensor Data/Testing'

# CSVPath1 = '/Users/jvivas/Documents/Aspen/GASCO/Sensor Data/To Be Processed'

# Windows Path
CSVPath1 = 'C:/Users/vivasj/Documents/Aspen/PTT GC/C002B/DATA/To be Processed'
FinalPath = 'C:/Users/vivasj/Documents/Aspen/PTT GC/C002B/DATA/Processed'

# CSVPath1 = 'C:/Mtell/Projects/XOM Baytown POC/Sensor Data/ToBeFormatted'

In [4]:
CSVFileList = []
NanPHDTagList = []
CSVFileListAll = listdir(CSVPath1)

m = len(CSVFileListAll)

for i in range(m):
    fileNameStr = CSVFileListAll[i]
    fileStr = fileNameStr.split('.')[0]
    fileExt = fileNameStr.split('.')[1]
    if fileExt == "csv":
        CSVFileList.append(fileNameStr)

n = len(CSVFileList)

# Uncomment following cell for debugging purposes

In [5]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# np.set_printoptions(threshold=np.nan)

In [6]:
def GetFileSize(file):
    import os
    csvFileStatInfo = os.stat(file)
    csvFileSizeGB = csvFileStatInfo.st_size/1000000000
    return(csvFileSizeGB)    

In [7]:
def RenameColumn(df2):
# Ranane Date and time column to DATETIME
    new_cols = ['DATETIME']
    df2.rename(columns=dict(zip(df2.columns[[0]], new_cols)),inplace=True)
    return

In [8]:
def ApplyDateFormat(df2):

    # duplicate df2 to apply date and time format 
    # df2 = df2.copy()    
    # Change new datetime column to datetime format
    df2['DATETIME'] = pd.to_datetime(df2['DATETIME'])    
    # Change datetime column format to look like 01/31/2015 0:00:00
    df2['DATETIME'] = df2['DATETIME'].dt.strftime('%m/%d/%Y %H:%M:%S')
    return

In [9]:
def SplitCSVFile_GetStrings(CSVFileWithPath, CSVFile):
    chunksize = 25000

    i = 0
    j = 1
    
    SensorStringListAllChunks = []
    StringListForAllSensors = []
    StringListForEachChunk = []
        
    print ('Loading ' + CSVFile + ' file')
    for df in pd.read_csv(CSVFileWithPath, chunksize=chunksize, iterator=True, low_memory=False):
        # df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
        df.index += j
        i+=1
        j = df.index[-1] + 1
        h = 0
        
        # Rename date and time column
        print ('Renaming date and time column')
        RenameColumn(df)

        # Apply Date and time format to dataframe
        print ('Applying date and time format')
        ApplyDateFormat(df)

        # Code to do Transposing
        # Create two dataframes df1 only with tags and descriptions. df2 tag with values
        if i == 1:
            df1 = df[0:1]  # FIRST ROW
            # Indexing dataframe df1
            df1 = df1.set_index('DATETIME')
        else:
            if i > 1:
                df1 = df1
                
        df2 = df[1:len(df)]  # SECOND TO LAST ROW

        # Indexing dataframe df2
        df2 = df2.set_index('DATETIME')
        
        # Export CSV chunk for each loop
        print ('Exporting chunk' + str(i))
        df.to_csv(CSVFileWithPath.replace('.csv', '') + '_chunk_ ' + str(i) + '.csv', index=True)
        
        print ('Getting the list for chunk' + str(i))
        StringListForEachChunk = ExtractStrings(df1, df2)
        
        print ('These are all the strings found in the chunk ' + str(i) + ':' + str(StringListForEachChunk))
        
        SensorStringListAllChunks.extend(StringListForEachChunk)

    SensorStringListAllChunks = list(set(SensorStringListAllChunks))
    return(SensorStringListAllChunks)

In [10]:
def ExtractStrings(df2_1, df2_2):
        
    print ('Converting df to numeric')
    # Convert columns to numbers those that has string wil be converted to numpy null = NaN
    df2_with_nan = df2_2.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    # print (df2_2.head())
    # print (df2_with_nan.head())
    # Ge the list of sensors with Strings
    df3 = pd.DataFrame(df2_with_nan.isnull().any(axis=0))
    df3 = df3.reset_index()
    SensorsWithStrings = df3[df3[0]==True]['index'].tolist()
    
    # Replace nan with 'null' and create new df
    df2_with_null = df2_with_nan.fillna(value='null')
    
    # Here we filter the columns that are objects and create a dataframe with those columns
    colsObject = df2_2.select_dtypes(include=['object']).columns
    TotalColumns = df2_2.columns
    
    print ('Total columns with text: ' + str(TotalColumns))
    print ('Sensors with text in their Values: ' + str(SensorsWithStrings))

    AllSensorsStringList = []
    
    if len(SensorsWithStrings) == 0:
        print ('No strings in this dataframe')
    else:
        # print (str(len(SensorsWithStrings)) + ' columns contain strings out of ' + str(len(TotalColumns)) + ' columns')
        # if (len(SensorsWithStrings)/len(TotalColumns)) > 0.5:
            # print ('this process will take some time depending of the size of the file and the PC resources')

        # Dropping DATETIME index to merge df1 and df2
        df2_1 = df2_1.reset_index(drop=False)
        df2_2 = df2_2.reset_index(drop=False)
        
        # Variable initialization
        j = 0

        StringListForCurrentSensor = []

        # Create dictionary from dataframe columns (sensors) that have strings only
        SensorDictionary = {}.fromkeys(SensorsWithStrings, [])
        
        # Loop to go thru each column and convert the characters string to numbers    
        for j in range(len(SensorsWithStrings)):
            # from IPython.core.debugger import Tracer
            # Tracer()() #this one triggers the debugger

            # iterate thru each column in the dataframe
            # for j in range(len(list(SensorDictionary))):
            # update sensor name for each column
            Sensor = list(SensorDictionary)[j]

            print ('Processing Sensor: ' + Sensor)
            # Clear List of String for each Sensor
            SensorStringResult = []

            # Get rows that have null in the actual sensor column
            result = df2_with_nan[df2_with_nan[Sensor].isnull()][Sensor]

            # Convert list of nulls to dataframe and reset the datetime index
            result_df = pd.DataFrame(result)
            result_df = result_df.reset_index()
            
            # print ('printing result_df')
            # print (result_df.head())
            # print ('printing df2_2')
            # print (df2_2.head())
            # debugging
            # print (result_df)
            # debugging

            # Filtering rows with String for each sensor
            SensorStringResult = df2_2[df2_2['DATETIME'].isin(result_df.loc[:,"DATETIME"].values.tolist())][Sensor]

            # Adding Strings to Sensor in Sensor Dictionary
            # I think that when due to a bug the dictionary become too big it stops growing...
            # SensorDictionary[Sensor] = list(set(SensorStringResult))
            StringListForCurrentSensor = list(set(SensorStringResult))
            
            # Debugging
            # print (list(set(SensorStringResult)))
            # Debugging
            
            # print ('Sensor: ' + Sensor + ' Processed')

            # Adding String for Sensor to General String List
            AllSensorsStringList.extend(StringListForCurrentSensor)

            # Removing Duplicate Strings
            AllSensorsStringList = list(set(AllSensorsStringList))
            
            print ('These are all the strings found in Sensor ' + Sensor + ': ' + str(StringListForCurrentSensor))
    return (AllSensorsStringList)

In [11]:
def SetIndex(df2):
        # Code to do Transposing
    # Create two dataframes df1 only with tags and descriptions. df2 tag with values
    df2_1 = df2[0:1]  # FIRST ROW
    df2_2 = df2[1:len(df2)]  # SECOND TO LAST ROW
    
    # Indexing dataframe df1
    df2_1 = df2_1.set_index('DATETIME')

    # Indexing dataframe df2
    df2_2 = df2_2.set_index('DATETIME')
    return(df2_1, df2_2)

In [12]:
def LoadCSV(FileAndPath, file):
    print ('Loading ' + file)
    df2 = pd.read_csv(FileAndPath, low_memory=False)
    return(df2)

In [13]:
def FormatToPrevise(df2_1, df2_2):
    # Dropping DATETime index to merge df1 and df2
    df2_1 = df2_1.reset_index(drop=False)
    df2_2 = df2_2.reset_index(drop=False)

    # Converting Historian files to VTQ format (DATETime, TAGNAME, DESCRIPTION, VALUE)
    mdf = pd.merge(pd.melt(df2_1, id_vars=['DATETIME'], var_name='TAGNAME',
                           value_name='DESCRIPTION')[['TAGNAME', 'DESCRIPTION']],
                   pd.melt(df2_2, id_vars=['DATETIME'], var_name='TAGNAME',
                           value_name='VALUE'),
                   on=['TAGNAME'])

    # Sort columns by VTQ format
    mdf = mdf[['DATETIME', 'TAGNAME', 'DESCRIPTION', 'VALUE']]
    
    return (mdf)

In [14]:
def SplitPreviseFormatCSVFile(mdf):
    # Exporting PHD Tag CSV file
    i = 0
    rows = 1000000
    totalRows = len(mdf)
    loops = math.ceil(totalRows/rows) + 1

    if totalRows > 1000000:
        for j in range(loops): #need to round this
            j = j + 1
            print (str(j))
            a = (rows*j) - rows
            if totalRows <= rows:
                b = totalRows
                print('Exporting ' + str(CSVFileList[i].replace('.csv', '')) + ' Historian File')
                print("")
                mdf[a:b].to_csv(FinalPath + '/' + str(CSVFileList[i].replace('.csv', '')) + '_Formatted.csv', index=False)
            else:
                if (rows*j) >= totalRows:
                    b = totalRows
                    print('Exporting ' + str(CSVFileList[i].replace('.csv', '')) + ' chunk' + str(j) + ' Historian File')
                    print("")
                    mdf[a:b].to_csv(FinalPath + '/' + str(CSVFileList[i].replace('.csv', '')) + '_Formatted_chunk' + str(j) + '.csv', index=False)
                else:
                    b = (rows*j) - 1
                    print('Exporting ' + str(CSVFileList[i].replace('.csv', '')) + ' chunk' + str(j) + ' Historian File')
                    print("")
                    mdf[a:b].to_csv(FinalPath + '/' + str(CSVFileList[i].replace('.csv', '')) + '_Formatted_chunk' + str(j) + '.csv', index=False)
    else:
        mdf.to_csv(FinalPath + '/' + str(CSVFileList[0].replace('.csv', '')) + '_Formatted' + '.csv', index=False)
    return

# This is the Main Code to execute

In [15]:
CSVFile = CSVFileList[0]
CSVFileWithPath = CSVPath1 + "/" + CSVFileList[0]

csvFileSizeGB = GetFileSize(CSVFileWithPath)

df2 = LoadCSV(CSVFileWithPath, CSVFile)

Loading 200C2_Erroring_3Columns.csv


In [17]:
# Rename date and time column
print ('Renaming date and time column')
RenameColumn(df2)
# Apply Date and time format to dataframe
print ('Applying date and time format')
ApplyDateFormat(df2)

# Apply Index and create two dataframes
df2_1, df2_2 = SetIndex(df2)

Renaming date and time column
Applying date and time format


In [21]:
df2_1.head()

,A1-200TI727HA.PV,A1-200TI727HB.PV,905II259BI.PV,905II259B.CPV
DATETIME,,,,
NaT,Motor bearing Temp,Motor bearing Temp NDE(SPARE),,NET GAS COMP 200-C2B


In [18]:
df2_2.head()

,A1-200TI727HA.PV,A1-200TI727HB.PV,905II259BI.PV,905II259B.CPV
DATETIME,,,,
01/01/2014 00:00:00,No Data,No Data,277.2757568,274.1443481
01/01/2014 00:05:00,No Data,No Data,269.463562,272.9598083
01/01/2014 00:10:00,No Data,No Data,261.2208862,253.5136414
01/01/2014 00:15:00,No Data,No Data,268.6708069,270.455658
01/01/2014 00:20:00,No Data,No Data,276.6998901,269.4458313


In [19]:
df2_with_nan = df2_2.apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [20]:
df2_with_nan.isnull().any(axis=0)

A1-200TI727HA.PV    True
A1-200TI727HB.PV    True
905II259BI.PV       True
905II259B.CPV       True
dtype: bool

In [22]:
df3 = pd.DataFrame(df2_with_nan.isnull().any(axis=0))

In [23]:
df3

,0
A1-200TI727HA.PV,True
A1-200TI727HB.PV,True
905II259BI.PV,True
905II259B.CPV,True


In [24]:
df3 = df3.reset_index()

In [25]:
df3

,index,0
0,A1-200TI727HA.PV,True
1,A1-200TI727HB.PV,True
2,905II259BI.PV,True
3,905II259B.CPV,True


In [26]:
SensorsWithStrings = df3[df3[0]==True]['index'].tolist()
SensorsWithStrings

['A1-200TI727HA.PV', 'A1-200TI727HB.PV', '905II259BI.PV', '905II259B.CPV']

In [27]:
df2_with_null = df2_with_nan.fillna(value='null')

In [28]:
colsObject = df2_2.select_dtypes(include=['object']).columns

In [29]:
TotalColumns = df2_2.columns

In [30]:
AllSensorsStringList = []

In [31]:
df2_1 = df2_1.reset_index(drop=False)
df2_2 = df2_2.reset_index(drop=False)

In [33]:
StringListForCurrentSensor = []

# Create dictionary from dataframe columns (sensors) that have strings only
SensorDictionary = {}.fromkeys(SensorsWithStrings, [])


In [34]:
SensorDictionary

{'905II259B.CPV': [],
 '905II259BI.PV': [],
 'A1-200TI727HA.PV': [],
 'A1-200TI727HB.PV': []}

In [35]:
Sensor = list(SensorDictionary)[0]

In [36]:
Sensor

'A1-200TI727HA.PV'

In [38]:
Sensor = '905II259BI.PV'

In [41]:
df2_with_nan.tail()

,A1-200TI727HA.PV,A1-200TI727HB.PV,905II259BI.PV,905II259B.CPV
DATETIME,,,,
09/30/2016 23:40:00,NaN,51.999699,229.224930,229.250046
09/30/2016 23:45:00,NaN,51.999699,233.359192,233.421692
09/30/2016 23:50:00,NaN,51.999699,280.176605,280.745300
09/30/2016 23:55:00,NaN,51.999699,255.031662,253.203873
01/10/2016 00:00:00,NaN,51.999699,263.551819,263.930939


In [39]:
df2_with_nan[df2_with_nan[Sensor].isnull()][Sensor]

DATETIME
02/16/2014 05:20:00   NaN
04/07/2014 17:45:00   NaN
04/28/2014 09:55:00   NaN
04/28/2014 10:00:00   NaN
04/28/2014 10:05:00   NaN
04/28/2014 10:10:00   NaN
04/28/2014 10:15:00   NaN
04/28/2014 10:20:00   NaN
04/28/2014 10:25:00   NaN
04/28/2014 21:50:00   NaN
04/28/2014 21:55:00   NaN
04/28/2014 22:00:00   NaN
04/28/2014 22:05:00   NaN
04/28/2014 22:10:00   NaN
04/28/2014 22:15:00   NaN
04/28/2014 22:20:00   NaN
04/28/2014 22:25:00   NaN
04/28/2014 22:30:00   NaN
04/28/2014 22:35:00   NaN
04/28/2014 22:40:00   NaN
04/28/2014 22:45:00   NaN
04/28/2014 22:50:00   NaN
04/28/2014 22:55:00   NaN
04/28/2014 23:00:00   NaN
04/28/2014 23:05:00   NaN
04/28/2014 23:10:00   NaN
04/28/2014 23:15:00   NaN
04/28/2014 23:20:00   NaN
04/28/2014 23:25:00   NaN
04/28/2014 23:30:00   NaN
                       ..
04/28/2015 00:10:00   NaN
04/28/2015 00:15:00   NaN
04/28/2015 00:20:00   NaN
04/28/2015 00:25:00   NaN
04/28/2015 00:30:00   NaN
06/13/2015 11:55:00   NaN
06/13/2015 12:00:00   NaN
06/

In [57]:
df2_with_nan.head()

,A1-200TI727HA.PV,A1-200TI727HB.PV,905II259BI.PV,905II259B.CPV
DATETIME,,,,
01/01/2014 00:00:00,NaN,NaN,277.275757,274.144348
01/01/2014 00:05:00,NaN,NaN,269.463562,272.959808
01/01/2014 00:10:00,NaN,NaN,261.220886,253.513641
01/01/2014 00:15:00,NaN,NaN,268.670807,270.455658
01/01/2014 00:20:00,NaN,NaN,276.699890,269.445831


In [59]:
df2_with_nan[df2_with_nan[Sensor].isnull()][Sensor].to_csv(FinalPath + '/result.csv',index=True)

In [65]:
df3_with_nan = df2_with_nan.reset_index(drop=False)

In [66]:
df3_with_nan.head()

,DATETIME,A1-200TI727HA.PV,A1-200TI727HB.PV,905II259BI.PV,905II259B.CPV
0,01/01/2014 00:00:00,NaN,NaN,277.275757,274.144348
1,01/01/2014 00:05:00,NaN,NaN,269.463562,272.959808
2,01/01/2014 00:10:00,NaN,NaN,261.220886,253.513641
3,01/01/2014 00:15:00,NaN,NaN,268.670807,270.455658
4,01/01/2014 00:20:00,NaN,NaN,276.699890,269.445831


## Getting duplicate rows

In [72]:

df3_with_nan[df3_with_nan['DATETIME']=='02/05/2014 00:00:00']

,DATETIME,A1-200TI727HA.PV,A1-200TI727HB.PV,905II259BI.PV,905II259B.CPV
10080,02/05/2014 00:00:00,NaN,NaN,285.235565,274.915405
34847,02/05/2014 00:00:00,NaN,NaN,NaN,NaN


In [72]:
df3_with_nan[df3_with_nan['DATETIME']=='02/05/2014 00:00:00']

,DATETIME,A1-200TI727HA.PV,A1-200TI727HB.PV,905II259BI.PV,905II259B.CPV
10080,02/05/2014 00:00:00,NaN,NaN,285.235565,274.915405
34847,02/05/2014 00:00:00,NaN,NaN,NaN,NaN


In [69]:
df2_with_nan[df2_with_nan[Sensor].isnull()][Sensor]

DATETIME
02/16/2014 05:20:00   NaN
04/07/2014 17:45:00   NaN
04/28/2014 09:55:00   NaN
04/28/2014 10:00:00   NaN
04/28/2014 10:05:00   NaN
04/28/2014 10:10:00   NaN
04/28/2014 10:15:00   NaN
04/28/2014 10:20:00   NaN
04/28/2014 10:25:00   NaN
04/28/2014 21:50:00   NaN
04/28/2014 21:55:00   NaN
04/28/2014 22:00:00   NaN
04/28/2014 22:05:00   NaN
04/28/2014 22:10:00   NaN
04/28/2014 22:15:00   NaN
04/28/2014 22:20:00   NaN
04/28/2014 22:25:00   NaN
04/28/2014 22:30:00   NaN
04/28/2014 22:35:00   NaN
04/28/2014 22:40:00   NaN
04/28/2014 22:45:00   NaN
04/28/2014 22:50:00   NaN
04/28/2014 22:55:00   NaN
04/28/2014 23:00:00   NaN
04/28/2014 23:05:00   NaN
04/28/2014 23:10:00   NaN
04/28/2014 23:15:00   NaN
04/28/2014 23:20:00   NaN
04/28/2014 23:25:00   NaN
04/28/2014 23:30:00   NaN
                       ..
04/28/2015 00:10:00   NaN
04/28/2015 00:15:00   NaN
04/28/2015 00:20:00   NaN
04/28/2015 00:25:00   NaN
04/28/2015 00:30:00   NaN
06/13/2015 11:55:00   NaN
06/13/2015 12:00:00   NaN
06/

In [42]:
result = df2_with_nan[df2_with_nan[Sensor].isnull()][Sensor]

In [43]:
result

DATETIME
02/16/2014 05:20:00   NaN
04/07/2014 17:45:00   NaN
04/28/2014 09:55:00   NaN
04/28/2014 10:00:00   NaN
04/28/2014 10:05:00   NaN
04/28/2014 10:10:00   NaN
04/28/2014 10:15:00   NaN
04/28/2014 10:20:00   NaN
04/28/2014 10:25:00   NaN
04/28/2014 21:50:00   NaN
04/28/2014 21:55:00   NaN
04/28/2014 22:00:00   NaN
04/28/2014 22:05:00   NaN
04/28/2014 22:10:00   NaN
04/28/2014 22:15:00   NaN
04/28/2014 22:20:00   NaN
04/28/2014 22:25:00   NaN
04/28/2014 22:30:00   NaN
04/28/2014 22:35:00   NaN
04/28/2014 22:40:00   NaN
04/28/2014 22:45:00   NaN
04/28/2014 22:50:00   NaN
04/28/2014 22:55:00   NaN
04/28/2014 23:00:00   NaN
04/28/2014 23:05:00   NaN
04/28/2014 23:10:00   NaN
04/28/2014 23:15:00   NaN
04/28/2014 23:20:00   NaN
04/28/2014 23:25:00   NaN
04/28/2014 23:30:00   NaN
                       ..
04/28/2015 00:10:00   NaN
04/28/2015 00:15:00   NaN
04/28/2015 00:20:00   NaN
04/28/2015 00:25:00   NaN
04/28/2015 00:30:00   NaN
06/13/2015 11:55:00   NaN
06/13/2015 12:00:00   NaN
06/

In [44]:
result_df = pd.DataFrame(result)

In [45]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10216 entries, 02/16/2014 05:20:00 to 08/15/2016 21:35:00
Data columns (total 1 columns):
905II259BI.PV    0 non-null float64
dtypes: float64(1)
memory usage: 479.6+ KB


In [47]:
result_df.tail()

,905II259BI.PV
DATETIME,
08/15/2016 20:40:00,NaN
08/15/2016 20:45:00,NaN
08/15/2016 20:50:00,NaN
08/15/2016 21:05:00,NaN
08/15/2016 21:35:00,NaN


In [48]:
result_df = result_df.reset_index()

In [49]:
result_df.tail()

,DATETIME,905II259BI.PV
10211,08/15/2016 20:40:00,NaN
10212,08/15/2016 20:45:00,NaN
10213,08/15/2016 20:50:00,NaN
10214,08/15/2016 21:05:00,NaN
10215,08/15/2016 21:35:00,NaN


In [54]:
result_df['DATETIME'].to_csv(FinalPath + '/result_df.csv',index=False)

In [ ]:
Sensor

In [50]:
df2_2[df2_2['DATETIME']=='08/15/2016 21:35:00'][Sensor]

271551    Bad
Name: 905II259BI.PV, dtype: object

In [ ]:
df2_2.loc[:, Sensor]
# df2.loc["Alaska":"Arkansas","2005":"2007"]

In [53]:
thelist = result_df.loc[:,'DATETIME'].values.tolist()
thelist

['02/16/2014 05:20:00',
 '04/07/2014 17:45:00',
 '04/28/2014 09:55:00',
 '04/28/2014 10:00:00',
 '04/28/2014 10:05:00',
 '04/28/2014 10:10:00',
 '04/28/2014 10:15:00',
 '04/28/2014 10:20:00',
 '04/28/2014 10:25:00',
 '04/28/2014 21:50:00',
 '04/28/2014 21:55:00',
 '04/28/2014 22:00:00',
 '04/28/2014 22:05:00',
 '04/28/2014 22:10:00',
 '04/28/2014 22:15:00',
 '04/28/2014 22:20:00',
 '04/28/2014 22:25:00',
 '04/28/2014 22:30:00',
 '04/28/2014 22:35:00',
 '04/28/2014 22:40:00',
 '04/28/2014 22:45:00',
 '04/28/2014 22:50:00',
 '04/28/2014 22:55:00',
 '04/28/2014 23:00:00',
 '04/28/2014 23:05:00',
 '04/28/2014 23:10:00',
 '04/28/2014 23:15:00',
 '04/28/2014 23:20:00',
 '04/28/2014 23:25:00',
 '04/28/2014 23:30:00',
 '04/28/2014 23:35:00',
 '04/28/2014 23:40:00',
 '04/28/2014 23:45:00',
 '04/28/2014 23:50:00',
 '04/28/2014 23:55:00',
 '04/29/2014 00:00:00',
 '04/29/2014 00:05:00',
 '04/29/2014 00:10:00',
 '04/29/2014 00:15:00',
 '04/29/2014 00:20:00',
 '04/29/2014 00:25:00',
 '04/29/2014 00:

In [52]:
df2_2[df2_2['DATETIME'].isin(result_df.loc[:,'DATETIME'].values.tolist())][Sensor]

1153      241.1456604
1154      235.0980988
1155       263.460968
1156      263.7768555
1157      262.3990479
1158       272.521637
1159      260.5787354
1160      258.8544312
1161      237.4663544
1162      246.8421478
1163      263.1885681
1164      270.0816956
1165      256.6778564
1166      275.1208191
1167      244.5154724
1168      263.1025696
1169      270.1039429
1170      258.8836365
1171      272.8731689
1172      254.2337799
1173      276.7945557
1174      262.9949951
1175      239.6256256
1176      249.6693878
1177      240.0330963
1178       242.628891
1179      242.3943481
1180      259.7544861
1181      249.1735535
1182      269.3968811
             ...     
138816       Shutdown
138817       Shutdown
138818       Shutdown
138819       Shutdown
138820       Shutdown
152204       Shutdown
152205       Shutdown
152206       Shutdown
159160       Shutdown
215033       Shutdown
238641            Bad
238642            Bad
238643            Bad
238644            Bad
266801    

In [ ]:
CSVFile = CSVFileList[0]
CSVFileWithPath = CSVPath1 + "/" + CSVFileList[0]

csvFileSizeGB = GetFileSize(CSVFileWithPath)

if csvFileSizeGB > 1:
    StringListForAllSensors = SplitCSVFile_GetStrings(CSVFileWithPath, CSVFile)
    print ('These are all the strings found in the file: ' + str(StringListForAllSensors))
else:
    
    df2 = LoadCSV(CSVFileWithPath, CSVFile)
    
    # Rename date and time column
    print ('Renaming date and time column')
    RenameColumn(df2)
    # Apply Date and time format to dataframe
    print ('Applying date and time format')
    ApplyDateFormat(df2)
    
    # Apply Index and create two dataframes
    df2_1, df2_2 = SetIndex(df2)
    
    print ('Extracting strings from csv file')
    StringListForAllSensors = ExtractStrings(df2_1, df2_2)
    
    print ('These are all the strings found in the csv file: ' + str(StringListForAllSensors))

# Convert all strings to a Dictionary
StringListDict = {}.fromkeys(StringListForAllSensors, 'null')

In [ ]:
# CSVFile = CSVFileList[0]
# CSVFileWithPath = CSVPath1 + "/" + CSVFileList[0]

# StringListForAllSensors = SplitCSVFile_GetStrings(CSVFileWithPath, CSVFile)

# Run following line to see the list of Strings
## Copy the result and paste it in the following line of code

In [ ]:
StringListDict

# Now we can replace string with null or a number
## I have assigned null to all strings by default all strings are replaced with null. If the strings need to be replaced with a number please replace the null with your desired number
###Example
###{'String 1': 'null',
    'String 2': 1,
    'String 3': 0}

### The replace code is still Work in Progress for files bigger than 1 GB
#### What to do
##### 1. Since the code does not load the whole CSV but splits it. The code needs to open each splitted file and do the replace
##### 2. Once text is replaced the CSV file will be updated and load the next
##### 3. Once all csv file are processed We need to convert them to Previse Format

In [ ]:
# Replacing known test to null
df2_2.replace(
{'Bad': 0,
 'CLOSE': 0,
 'Comm Fail': 'null',
 'Configure': 'null',
 'Error': 'null',
 "Exception of type 'System.OutOfMemoryException' was thrown.": 'null',
 'Failed': 'null',
 'I/O Timeout': 'null',
 'Intf Shut': 'null',
 'MEM_ERR': 'null',
 'ON': 1,
 'OPEN': 1,
 'Off': 0,
 'RUN': 1,
 'STOP': 0,
 'Scan Off': 'null',
 'Scan Timeout': 'null'}
, inplace=True, regex=True)

# Now we convert the data to linear format Previse

## -----------------------------------------------------------------------------------------
## The format and Split will be done in the same step (this is to be done for files bigger than 1GB)
### The code will loop thru each chunk and will Format to Previse then will Split the file
### Each file will be a sensor
### If the sensor csv file has more than 1MM rows the code will split that file also
## -----------------------------------------------------------------------------------------

## The FormatToPrevise code is still Work in Progress for files bigger than 1 GB
### What to do
#### 1. Since the code does not load the whole CSV but splits it. The code needs to open each splitted file and do the Formatting
#### 2. The code will loop thru each Splitted CSV file and format each one of them
#### 3. Once formatted the code will split the file into 1MM rows per CSV

In [ ]:
mdf = FormatToPrevise(df2_1,df2_2)

# Run following line to see the TOP 25 rows for the new Previse Format

In [ ]:
mdf.head(25)

In [ ]:
df2.info()

# Here we split the CSV data by file and each file will contain 1 million rows

### The SplitPreviseFormatCSVFile code is still Work in Progress for files bigger than 1 GB
### What to do

### Improvement
#### 1. I will update the code to create a CSV file per Sensor and if the sensor has more than 1MM rows it will splitted

In [ ]:
SplitPreviseFormatCSVFile(mdf)